<a href="https://colab.research.google.com/github/Asmaaelkashef/Flower_detection_model/blob/main/Flowers_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Flowers Recognition "ResNet pre-trained model"

In [ ]:
!pip install tensorflow kagglehub


In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input

train_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True )

val_aug = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [15]:
target_size = (224, 224)
batch_size = 32

train_data = train_aug.flow_from_directory(
    "/kaggle/input/flowers-recognition/flowers",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = val_aug.flow_from_directory(
    "/kaggle/input/flowers-recognition/flowers",
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [27]:
print(train_data.class_indices)

{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [16]:
from tensorflow.keras.applications import ResNet50
resnet_base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
resnet_base.trainable = False

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers



model = Sequential()
model.add(resnet_base)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.4))

model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(64))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

In [20]:
model.fit(train_data, validation_data=val_data, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 113s 832ms/step - accuracy: 0.5968 - loss: 1.0642 - val_accuracy: 0.8849 - val_loss: 0.3776
Epoch 2/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 58s 531ms/step - accuracy: 0.8363 - loss: 0.4780 - val_accuracy: 0.8965 - val_loss: 0.3079
Epoch 3/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 57s 526ms/step - accuracy: 0.8698 - loss: 0.3802 - val_accuracy: 0.8942 - val_loss: 0.2919
Epoch 4/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 81s 519ms/step - accuracy: 0.8810 - loss: 0.3238 - val_accuracy: 0.8907 - val_loss: 0.3139
Epoch 5/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 57s 527ms/step - accuracy: 0.9000 - loss: 0.2954 - val_accuracy: 0.9058 - val_loss: 0.2907
Epoch 6/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 58s 532ms/step - accuracy: 0.9126 - loss: 0.2634 - val_accuracy: 0.9105 - val_loss: 0.2731
Epoch 7/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 58s 532ms/step - accuracy: 0.9247 - loss: 0.2310 - val_accuracy: 0.9198 - val_loss: 0.2652
Epoch 8/10
109/109 ━━━━━━━━━━━━━━━━━━━━ 59s 546ms/step - accuracy: 0.9201 - loss: 

In [21]:
model.save("flower_model.keras")

In [22]:
from google.colab import files
files.download("flower_model.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,289,361 (96.47 MB)

 Trainable params: 566,917 (2.16 MB)

 Non-trainable params: 23,588,608 (89.98 MB)

 Optimizer params: 1,133,836 (4.33 MB)

In [25]:
print(train.class_indices)


{'flowers': 0}
